In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

### Load Samples

In [2]:
filename = "data_labeled_full.csv"

data_labeled = pd.read_csv(filename)

In [3]:
data_labeled.head()

,target,satellite,label,index,filename,view,period,t0,star_radius,k,aos,i,e,w,ldcs_1,ldcs_2,n_points_transit
0,HD102249,BHr,PC,0,0_curve_HD102249_BHr.npy,Curve,2.485603,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076
1,HD102249,BHr,PC,0,0_fold_HD102249_BHr.npy,PhaseFold,2.485603,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076
2,HD102249,BHr,PC,0,0_global_HD102249_BHr.npy,Global,2.485603,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076
3,HD102249,BHr,PC,0,0_local_HD102249_BHr.npy,Local,2.485603,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076
4,HD102249,BHr,PC,1,1_curve_HD102249_BHr.npy,Curve,2.486992,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076


### Selection

In [4]:
# using the same portion of [Yu (2019)]
total = 493+2155+13868
print(total)

# considering transit both PC's and EB's 
portion_transit = (493 + 2155)/total
portion_non_transit = 13868/total

print(portion_transit)
print(portion_non_transit)

print(portion_non_transit + portion_transit)

16516
0.1603293775732623
0.8396706224267377
1.0


In [5]:

count_transit = data_labeled[ data_labeled["label"] == "PC" ].shape[0]
count_non_transit = data_labeled[ data_labeled["label"] == "NOT_PC" ].shape[0]

#print("Number of transit samples: ", count_transit)
#print("Number of non-transit samples: ", count_non_transit)

print("Number of non-transit samples: ", count_non_transit//3)
print("Number of transit samples: ",  count_transit//4)
print("Total number of samples: ", count_non_transit//3 + count_transit//4)
print((count_transit//4)/(count_non_transit//3+count_transit//4 ))
print((count_non_transit//3)/(count_non_transit//3+count_transit//4  ))

#print(portion_transit/portion_non_transit)

Number of non-transit samples:  17872
Number of transit samples:  3351
Total number of samples:  21223
0.15789473684210525
0.8421052631578947


### Remove index
from manual selection

In [6]:
remove_indexs = [
    #------
    #page=0
    57,114,114,114,133,228,247,266,285,304,418,456,457,419,305,286,229,210,191,172,134,115,58,1,20,2,21,59,78,116,135,173,192,211,249,268,287,306,325,344,382,420,458,
    #------
    #page=1

    685,704,723,742,780,799,818,837,875,894,913,932,970,971,933,914,895,876,857,838,819,800,781,743,724,705,686,684,703,722,741,779,798,817,855,874,893,912,931,969,
    #------
    #page=2

    1351,1655,1883,1881,1349,1350,1863,1882,
    #------
    #page=3

    1900,1938,2014,2261,2337,2338,2262,2015,1901,1902,1940,1978,2016,2358,2339,2263,
    #------
    #page=4

    2642,2699,3079,3098,3136,3155,3137,3099,3080,2700,2643,2698,3078,3097,3135,3154,
    #------
    #page=5

    3156,3175,3194,3213,3232,3327,3422,3441,3631,3612,3726,3745,3783,3781,3743,3724,3629,3610,3591,3420,3439,3268,3230,3211,3192,3173,3174,3193,3212,3231,3421,3440,3478,3611,3630,3725,3782,
    #------
    #page=6

    4047,4123,4180,4256,4294,4389,4408,4409,4390,4333,4295,4181,4124,3973,4030,4125,4163,4144,4182,4201,4220,4258,4296,4334,4353,4391,4410,4372,
    #------
    #page=7

    4865,4866,4864,
    #------
    #page=8

    5094,5208,5227,5284,5322,5455,5453,5358,5320,5282,5263,5225,5206,5188,5207,5226,5264,5283,5302,5321,
    #------
    #page=9

    5757,6175,6194,6213,6232,6251,6270,6289,6308,6327,6309,6290,6271,6252,6214,6195,6176,5758,5759,6177,6196,6215,6234,6253,6272,6291,6310,
    #------
    #page=10

    6328,6537,6689,6879,6936,6937,6880,6538,6424,6329,6422,6517,6536,6745,6878,6935,
    #------
    #page=11

    7032,7013,7488,7486,7467,7011,7012,7373,7392,7468,7487,
    #------
    #page=12

    7600,7733,7752,7771,7809,7904,7961,7980,7999,8018,8075,8227,8076,8019,7981,7962,7905,7848,7810,7772,7753,7734,7601,7602,7735,7754,7773,7811,7849,7906,7963,8001,8020,8077,
    #------
    #page=13

    8228,8247,8266,8304,8342,8361,8380,8608,8741,8760,8779,8798,8817,8836,8855,8837,8818,8799,8780,8761,8742,8609,8533,8514,8495,8476,8457,8438,8419,8400,8381,8362,8343,8305,8267,8248,8229,8303,8341,8360,8379,8398,8417,8436,8455,8474,8493,8512,8531,8550,8607,8683,8740,8759,8778,8797,8816,8835,8854,
    #------
    #page=14

    8856,8875,8894,8913,8932,8951,8970,8989,9008,9065,9046,9179,9217,9255,9312,9369,9464,9462,9405,9406,9367,9329,9310,9253,9215,9139,9063,9044,9006,9025,8930,8911,8892,8873,8874,8893,8912,8931,8969,8968,9007,9026,9045,9064,9368,9406,9425,9463,9272,
    #------
    #page=15

    9500,9519,9557,9595,9614,9633,9652,9728,9709,9804,9842,9861,9880,9899,9956,9976,9843,9710,9634,9615,9596,9577,9558,9520,9501,9502,9521,9559,9578,9597,9616,9635,9711,9730,9749,9768,9844,9863,9882,9901,9958,9977,
    #------
    #page=16

    10242,10299,10337,10356,10394,10413,10432,10470,10546,10736,10471,10433,10414,10395,10357,10338,10243,10186,10241,10317,10300,10336,10355,10374,10393,10412,10431,10450,10469,10545,10735,10547,10737,
    #------
    #page=17

    10775,10794,10813,10832,10965,11003,11060,11098,11193,11212,11288,11307,11326,11345,11364,11381,11362,11343,11324,11305,11286,11210,11191,11096,11058,11001,10963,10906,10830,10811,10792,10774,10793,10812,10831,10926,10964,11002,11097,11192,11211,11287,11306,11325,11344,
    #------
    #page=18

    11419,11476,11533,11571,11609,11628,11647,11666,11685,11704,11723,11742,11761,11780,11799,11818,11856,11875,11894,11913,11932,11951,11970,12008,12027,12009,11800,11781,11762,11743,11724,11705,11686,11667,11648,11610,11572,11573,11611,11630,11649,11668,11687,11706,11725,11744,11763,11782,11801,11858,11877,11896,11915,11934,11953,11972,12010,
    #------
    #page=19

    12028,12047,12066,12104,12123,12142,12180,12199,12218,12275,12332,12408,12503,12522,12560,12579,12636,12637,12580,12561,12523,12504,12409,12333,12295,12276,12219,12200,12181,12143,12105,12086,12067,12048,12029,12046,12065,12103,12122,12141,12160,12179,12198,12217,12274,12293,12331,12407,12502,12521,12559,12578,12635,
    #------
    #page=20

    12694,12941,13207,13226,13245,13243,13224,13205,12939,12883,12940,13206,13225,13244,
    #------
    #page=21

    13300,13471,13851,13870,13889,13908,13927,13909,13871,13890,13852,13472,13301,13302,13473,13872,13891,13910,
    #------
    #page=22

    13928,13947,13966,14080,14194,14213,14232,14384,14403,14422,14441,14460,14479,14517,14536,14555,14537,14518,14480,14461,14442,14499,14516,14498,14423,14404,14385,14402,14233,14214,14195,14081,13967,13948,13929,13946,13965,14079,14193,14212,14231,14421,14440,14459,14478,14497,14535,14554,
    #------
    #page=23

    14556,14575,14632,14670,14689,14708,14727,14746,14765,14803,14860,14936,15069,15107,15183,15143,15105,15067,14934,14858,14820,14801,14763,14744,14725,14706,14687,14668,14630,14611,14592,14573,14574,14593,14612,14631,14669,14688,14707,14726,14745,14764,14802,14859,14935,15068,14821,15106,15144,15163,15182,15181,15164,15145,15162,14594,
    #------
    #page=24

    15200,15219,15238,15276,15352,15390,15466,15485,15504,15523,15637,15656,15675,15713,15732,15808,15827,15809,15733,15714,15657,15638,15524,15505,15486,15467,15391,15353,15277,15239,15220,15201,15202,15221,15240,15278,15354,15373,15392,15411,15430,15468,15487,15506,15525,15563,15582,15639,15658,15696,15695,15639,15734,15715,15810,15580,
    #------
    #page=25

    15828,15847,15866,15904,15942,15961,15980,15999,16018,16037,16056,16075,16303,16360,16398,16417,16436,16437,16418,16399,16342,16323,16304,16076,16057,16038,16019,16000,15981,15962,15943,15829,15848,15867,15846,15865,15960,15941,15979,15998,16036,16055,16074,16093,16283,16302,16340,16359,16397,16416,16435,
    #------
    #page=26

    16475,16570,16627,16665,16684,16703,16741,16798,16950,17045,17043,16948,16910,16739,16701,16663,16682,16683,16625,16569,16626,16664,16702,16740,16949,17025,17044,
    #------
    #page=27

    17100,17176,17290,17309,17328,17347,17537,17556,17613,17632,17651,17670,17708,17709,17671,17557,17177,17101,17102,17178,17311,17330,17349,17539,17558,17615,17634,17672,17710,17691,17614,
    #------
    #page=28

    17766,17785,17975,17994,18013,18032,18051,18070,18089,18108,18127,18146,18165,18184,18203,18222,18241,18260,18279,18298,18317,18336,18355,18337,18318,18299,18280,18261,18242,18223,18204,18185,18166,18147,18128,18109,18090,18071,18052,18033,18014,17995,17976,17974,17862,17767,17786,17765,17784,17993,18012,18031,18050,18069,18088,18107,18126,18145,18164,18183,18202,18221,18240,18259,18278,18297,18316,18335,18354,
    #------
    #page=29


    18375,18394,18413,18432,18451,18470,18489,18508,18527,18546,18565,18584,18622,18641,18660,18679,18736,18755,18774,18793,18850,18888,18907,18945,18964,18983,18981,18962,18943,18905,18886,18848,18791,18772,18753,18734,18677,18658,18639,18620,18582,18563,18544,18525,18506,18487,18449,18430,18411,18392,18373,18374,18393,18412,18450,18488,18507,18526,18545,18564,18583,18621,18640,18659,18678,18735,18773,18792,18811,18830,18849,18887,18906,18944,18963,18982,
    #------
    #page=30

    19000,19209,19285,19342,19361,19380,19418,19437,19456,19475,19494,19532,19608,19627,19609,19533,19495,19476,19438,19419,19381,19362,19343,19344,19286,19002,19021,19097,19173,19211,19287,19382,19363,19401,19420,19439,19458,19477,19496,19534,19553,19001,
    #------
    #page=31

    19628,19685,19704,19723,19742,19761,19780,19799,19818,19837,19856,19894,19913,19989,20008,20237,20218,20180,20161,20178,20123,20066,20085,20083,20009,20026,19990,20007,19914,19895,19857,19838,19819,19800,19781,19781,19762,19743,19724,19705,19686,19667,19629,19684,19703,19722,19741,19760,19779,19798,19817,19836,19855,19893,19912,19988,20045,20216,20235,20254,20197,19666,
    #------
    #page=32

    20256,20275,20294,20313,20389,20408,20484,20503,20541,20560,20579,20598,20617,20636,20655,20674,20693,20712,20731,20750,20769,20788,20807,20826,20845,20864,20862,20843,20824,20805,20786,20767,20748,20729,20710,20691,20672,20653,20634,20615,20596,20577,20558,20539,20501,20482,20273,20292,20311,20349,20387,20406,20540,20559,20578,20597,20616,20635,20654,20673,20692,20711,20730,20749,20768,20787,20806,20825,20844,20863,20882,20881,
    #------
    #page=33

    20919,20995,21014,21128,21147,21186,21148,21129,21072,21071,21073,21053,21034,21015,21016,20996,20920,20921,20997,21035,21054,21073,21130,21149


]

In [7]:
print(len(remove_indexs))
print(len(np.unique(remove_indexs)))

1283
1277


In [8]:
print(len(np.unique(data_labeled[data_labeled["label"] == "PC"]["index"])))
      
      
selected_PC_indexs = []
for index in np.unique(data_labeled[data_labeled["label"] == "PC"]["index"]):
    if (index not in remove_indexs):
        selected_PC_indexs.append(index)
    
print(len(selected_PC_indexs))

print(len(np.unique(data_labeled[data_labeled["label"] == "PC"]["index"])) - len(selected_PC_indexs))


3351
2074
1277


In [9]:

n_pc = len(selected_PC_indexs)
n_not_pc = int(portion_non_transit * len(selected_PC_indexs) / portion_transit)

print(n_not_pc/(n_pc + n_not_pc))
print(n_pc/(n_pc + n_not_pc))

0.8396598376497874
0.1603401623502126


In [10]:
# se escoje aleatoriamente de not pc 
selected_not_PC_indexs = []
np.random.seed(952164)
for idx in np.random.choice(list(np.unique(data_labeled[data_labeled["label"] == "NOT_PC"]["index"])), n_not_pc, 
                            replace=False).tolist():
    selected_not_PC_indexs.append(idx)

In [11]:
print(len(selected_PC_indexs))
print(len(selected_not_PC_indexs))
print(len(selected_PC_indexs) + len(selected_not_PC_indexs))

print(len(selected_PC_indexs)/(len(selected_PC_indexs)+len(selected_not_PC_indexs)))
print(len(selected_not_PC_indexs)/(len(selected_PC_indexs)+len(selected_not_PC_indexs)))

2074
10861
12935
0.1603401623502126
0.8396598376497874


In [12]:
# el paso siguiente seria crear un dataframe con los selected_PC_indexs y con los selected_not_PC_index
data_labeled_selected = data_labeled.copy()
df1 = data_labeled_selected[data_labeled_selected['index'].isin(selected_PC_indexs)]
df2 = data_labeled_selected[data_labeled_selected['index'].isin(selected_not_PC_indexs)]
data_labeled_selected = pd.concat([df1, df2], ignore_index = True)

In [13]:
# check consistency
print(len(np.unique(data_labeled_selected['index'])) )
print(data_labeled_selected.head())
print(data_labeled_selected.tail())


12935
     target satellite label  index                   filename       view  \
0  HD102249       BHr    PC      0   0_curve_HD102249_BHr.npy      Curve   
1  HD102249       BHr    PC      0    0_fold_HD102249_BHr.npy  PhaseFold   
2  HD102249       BHr    PC      0  0_global_HD102249_BHr.npy     Global   
3  HD102249       BHr    PC      0   0_local_HD102249_BHr.npy      Local   
4  HD102249       BHr    PC     19  19_curve_HD102249_BHr.npy      Curve   

     period           t0  star_radius         k       aos         i    e    w  \
0  2.485603  1506.040641      3.47246  0.032584  3.100417  1.541473  0.0  0.0   
1  2.485603  1506.040641      3.47246  0.032584  3.100417  1.541473  0.0  0.0   
2  2.485603  1506.040641      3.47246  0.032584  3.100417  1.541473  0.0  0.0   
3  2.485603  1506.040641      3.47246  0.032584  3.100417  1.541473  0.0  0.0   
4  4.751788  1457.811459      3.47246  0.038840  4.774851  1.539857  0.0  0.0   

   ldcs_1  ldcs_2  n_points_transit  
0    0.32   

In [14]:
data_labeled_selected.to_csv('data_labeled_selected.csv', index = False)

### Split

In [15]:
from sklearn.model_selection import train_test_split


In [16]:
data_labeled_selected = pd.read_csv('data_labeled_selected.csv')


In [17]:
# check consistency
print(len(np.unique(data_labeled_selected['index'])) )
data_labeled_selected.head()

12935


,target,satellite,label,index,filename,view,period,t0,star_radius,k,aos,i,e,w,ldcs_1,ldcs_2,n_points_transit
0,HD102249,BHr,PC,0,0_curve_HD102249_BHr.npy,Curve,2.485603,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076
1,HD102249,BHr,PC,0,0_fold_HD102249_BHr.npy,PhaseFold,2.485603,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076
2,HD102249,BHr,PC,0,0_global_HD102249_BHr.npy,Global,2.485603,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076
3,HD102249,BHr,PC,0,0_local_HD102249_BHr.npy,Local,2.485603,1506.040641,3.47246,0.032584,3.100417,1.541473,0.0,0.0,0.32,0.07,2076
4,HD102249,BHr,PC,19,19_curve_HD102249_BHr.npy,Curve,4.751788,1457.811459,3.47246,0.038840,4.774851,1.539857,0.0,0.0,0.31,0.09,1487


In [18]:
test_size_ = 0.1
train_indexs, test_indexs = train_test_split(np.unique(data_labeled_selected["index"]), test_size=test_size_, random_state=53861793)


In [19]:
print(train_indexs.shape)
print(test_indexs.shape)

print(train_indexs.shape[0] / len(list(np.unique(data_labeled_selected["index"]))))
print(test_indexs.shape[0] / len(list(np.unique(data_labeled_selected["index"]))))

(11641,)
(1294,)
0.8999613451874758
0.10003865481252416


In [24]:
11641+1294

12935

In [20]:
data_train_full = data_labeled_selected[data_labeled_selected['index'].isin(train_indexs)]
data_test_full = data_labeled_selected[data_labeled_selected['index'].isin(test_indexs)]

In [21]:
print(data_train_full.shape)
print(data_test_full.shape)


(36761, 17)
(4118, 17)


In [22]:
data_test_without_label = data_test_full.copy()
data_test_without_label_ = data_test_without_label[(data_test_without_label["view"] == "Local")|(data_test_without_label["view"] == "Global")]
data_test_without_label_ = data_test_without_label_.drop(columns=['period', 't0', 'star_radius', 'k', 'aos', 'i', 'e', 'w', 'ldcs_1', 'ldcs_2', 'label'])
data_test_without_label_

,target,satellite,index,filename,view,n_points_transit
38,HD102249,BHr,97,97_global_HD102249_BHr.npy,Global,3322
39,HD102249,BHr,97,97_local_HD102249_BHr.npy,Local,3322
94,HD102776,BTr,361,361_global_HD102776_BTr.npy,Global,8113
95,HD102776,BTr,361,361_local_HD102776_BTr.npy,Local,8113
98,HD102776,BTr,362,362_global_HD102776_BTr.npy,Global,8113
...,...,...,...,...,...,...
40863,HD93194,BHr,21215,21215_local_HD93194_BHr.npy,Local,4724
40865,HD93194,BHr,21216,21216_global_HD93194_BHr.npy,Global,4724
40866,HD93194,BHr,21216,21216_local_HD93194_BHr.npy,Local,4724
40871,HD93194,BHr,21218,21218_global_HD93194_BHr.npy,Global,4724


In [23]:
# check consistency

print(data_test_without_label_.shape)
print(data_test_full[data_test_full["label"]=="PC"].shape[0]//4 + data_test_full[data_test_full["label"]=="NOT_PC"].shape[0]//3) # check consistency
print(data_test_without_label_.shape[0]//2) # check consistency

(2588, 6)
1294
1294


In [26]:
# save
data_train_full.to_csv('csv/data_train_full.csv', index = False)
data_test_full.to_csv('csv/data_test_full.csv', index = False)
data_test_without_label_.to_csv('csv/data_test_without_label.csv', index = False)